# Problem Description.

The goal of this project is to write an ASP encoding for
planning the movements of a group of robots that have to complete a set of jobs.

The robots navigate an undirected graph over a sequence of time steps that range from 0 to n.

Each robot belongs to one class and starts at some specific node in the graph.
At every time step, robots can either stay at their current node, 
or move to some adjacent node,
but two robots cannot occupy the same node simultaneously
(for example, robots r1 and r2 cannot be at node 1 at step 1)
or swap their positions
(for example, 
if at time point 0 robot r1 is at node 1 and robot r2 is at node 2, 
then at time point 1 it cannot be the case that r1 is at node 2 and r2 at node 1).

Each job belongs to one class, has some deadline, 
and consists of a sequence of operations that must be executed in order.
Each operation takes place at a designated node.

A solution to this problem consists of two parts:
* a sequence of valid movements for each robot, and
* an assignment of each job of a given class to exactly one robot of the same class.

A robot completes a job if it visits the places of the operations of the job in the right order, and it visits the place of the last operation before or at the deadline of the job.

A solution can assign more than one job to the same robot. 
In that case, the robot must execute the assigned jobs in some chosen order, 
one after the other.

Each operation takes one time step. 
This implies that if a job has two consecutive operations at the same node, 
then the robot assigned to that job must be at least two steps (not necessarily consecutive) 
at that node.
The same happens if a robot completes a job at a given place, 
and its next job starts at the same place.

A solution is optimal if it minimizes the overall number of robot movements.
We count one movement each time a robot moves to an adjacent cell, and
we count nothing when a robot remains at the same node.

# An Example

Consider an example of a corridor where there are two robots: 
* robot r1 of class c1 starting at node 1, and
* robot r2 of class c2 starting at node 5,

and two jobs: 
* job j1 of class c1 and deadline 11, that has three operations, 
  first at node 1, then at node 5, and finally at node 1,
* job j2 of class c2 and deadline 10, that has three operations, 
  first at node 5, then at node 1, and finally at node 5.
  
The example is represented in the next image.
The nodes of the graph are labeled on their top-left by their identifiers. 
Robots are painted in blue, 
jobs are described in a green rectangle, 
and their operations are attached to their corresponding nodes, 
also in green rectangles.
The yellow rectangle represents the clock.

<img src="img/gif_image_0_0.png" width="300" align="center">


To solve the problem, robot r1 has to be assigned to job j1, and robot r2 to job j2.
They both have to go to the other extreme of the corridor and come back to their initial position.
To do this, one of them has to use node 6 to give way to the other.
One solution is depicted in the following animated image. 
There is another solution, where robot r1 stays at the first step instead of staying at the second.
 
<img src="img/model-0.gif" width="300" align="center">

You may have to re-run the cell to start the animation again, or you can also open it directly [here](img/model-0.gif).

## F.A.Q.

Q: If a robot r1 is assigned two jobs j1 and j2, and the deadline of j1 is before the deadline of j2, must r1 complete j1 before completing j2? 

A: Not necessarily, it can first complete j2, and then complete j1, as long as they both meet their deadlines.

Q: Suppose a robot r1 is assigned two jobs j1 and j2, and this robot chooses to complete j1 before j2. If, while completing j1, the robot moves to the node of the first operation of j2, must r1 come back to this node after completing j1?

A: Yes. The robots must execute their jobs separately, one after the other. In other words, the nodes that a robot has visited while completing one job are irrelevant for the completion of the other jobs.

# Representation in ASP.

The examples are represented by facts over the following predicates:    
```
edge(N1,N2): there is an (undirected) edge between nodes N1 and N2
robot(R,C,N): robot R is of class C and starts at node N
job(J,C,D): job J is of class C and its deadline is D (D >= 0)
operation(J,O,N): J's operation O takes place at node N (O >= 1)
```
We assume that the operations O of a given job J
form a sequence 1, 2, 3, ...

Every problem instance also contains a constant declaration of the form
```
#const n=v.
```
for some integer `v`, 
that fixes the number of steps of the plan.

Our running example is represented as follows:
```
#const n=11.
edge(X,X+1) :- X=1..4.
edge(3,4+2).
robot(r1,c1,1).
robot(r2,c2,5).
job(j1,c1,11).
  operation(j1,1,1).
  operation(j1,2,5).
  operation(j1,3,1).
job(j2,c2,10).
  operation(j2,1,5).
  operation(j2,2,1).
  operation(j2,3,5).
```

A solution is represented by atoms over the following predicates:  
```
at(R,N,T): robot R is at node N at time step T
assign(R,J,I): assign to R the job J at index I (in the sequence of jobs assigned to R) (I >= 1)
```   
Predicate `at/3` specifies the positions of the robots, 
and predicate `assigns/3` specifies the assignment of jobs to robots. 
For every robot R, the indexes of the jobs assigned to it 
must form a sequence 1, 2, 3 ...

The solution show in the animation above consists of the following atoms:
```
at(r1,1,0) at(r1,2,1) at(r1,2,2) at(r1,3,3) at(r1,4, 4) at(r1,5, 5)
at(r1,4,6) at(r1,3,7) at(r1,6,8) at(r1,3,9) at(r1,2,10) at(r1,1,11)
at(r2,5,0) at(r2,4,1) at(r2,3,2) at(r2,6,3) at(r2,3, 4) at(r2,2, 5)
at(r2,1,6) at(r2,2,7) at(r2,3,8) at(r2,4,9) at(r2,5,10) at(r2,5,11)
assign(r1,j1,1) assign(r2,j2,1)
```

# Visualization

You can use clingraph to visualize the solutions:
* https://github.com/potassco/clingraph

In the folder `asp/solutions-for-clingraph` you can find the solutions for clingraph in ``json`` format. They show all atoms necessary for the visualization, not only those over predicates `at/3` and `assign/3`.

You can generate the animated gifs for our example using the following command:

In [ ]:
! cat asp/solutions-for-clingraph/corridor-01.json | clingraph --viz-encoding=asp/viz.lp  --out=animate --sort=asc-int  --engine=neato --prefix=viz_ --dir=img --name-format=model-{model_number}

The gifs can be found in the directory `img`.

Once you have an encoding to solve the problem, you can visualize the answers with the following command. We add the file ``asp/viz_show.lp`` to show some additional predicates, and options ``-q1,1,2 --outf=2`` to print in json format the optimal solutions and only those. Option ``--opt-mode=optN 0`` is used to enumerate all optimal answers.

In [ ]:
! clingo asp/robots.lp asp/viz_show.lp asp/instances/corridor-01.lp -q1,1,2 --outf=2 --opt-mode=optN 0 | clingraph --viz-encoding=asp/viz.lp  --out=animate --sort=asc-int  --engine=neato --prefix=viz_ --dir=img --name-format=model-{model_number}

# Framework.

The directory ``asp`` contains the files that you need for the project. In the directory ``asp/instances`` you can find the instances (our example is ``corridor-01.lp``) and in the directory ``asp/solutions`` you can find their solutions in ``json`` format. 

You have to submit a file named ``robots.lp``, included as a template in the directory ``asp``, that contains the following lines (and no more ``#show`` statements) so that in the output only occur the atoms of predicates ``at/3`` and ``assign/3``:

```
#show at/3
#show assign/3.
```

To run your encoding with our example, you can use this command:
* ``clingo asp/robots.lp asp/instances/corridor-01.lp``

You can use option `-t4` to try to improve the speed of clingo, 
but the test script does not use this option.

Option `-t4` runs clingo with 4 threads that use different parameters. 
This option usually improves the performance of the system,
but it makes the behavior of clingo non-deterministic.
In this way, different runs of the same command can compute answer sets in a different order.
But note that only the order of the answers varies:
if we compute all answer sets in two different runs, 
then the set of computed answers will always be the same.

To enumerate all optimal solutions, you can add the options ``--opt-mode=optN 0``,
but the test script only computes one optimal solution and does not use this option.

You can check if your encoding solves correctly all instances by running the ``Python`` script ``test.py`` as follows:
* ``python asp/test.py -e asp/robots.lp -i asp/instances -s asp/solutions -t 180 -opt``

In this case, the timeout for each instance is set to `180` seconds, but you can use any other value instead.

For help, type `python asp/test.py --help`.

We recommend you to work locally in your computer, using your own installation of ``clingo``.

For this, you can run the next cell to generate a zip file of this directory. The zip file will be stored in the parent directory with the name `robots.zip`. You can click on the folder symbol at the left of the screen to look for it and download it.

In [ ]:
import os
from shutil import make_archive
make_archive('../robots', 'zip', os.getcwd())

You can also run your encoding in the next cell. It is not recommended to work in this notebook at ``Binder``, but if you do it, remember to download the files that you modify to your computer, otherwise you will lose your changes.

In [ ]:
%%clingo 0 asp/instances/corridor-01.lp -

%
% Input:
%   #const n: the time steps range from 0 to n
%   edge(N1,N2): there is an (undirected) edge between nodes N1 and N2
%   robot(R,C,N): robot R is of class C and starts at node N
%   job(J,C,D): job J is of class C and its deadline is D 
%   operation(J,O,N): J's operation O takes place at node N (O >= 1)
%
% Output:
%   at(R,N,T): robot R is at node N at time step T
%   assign(R,J,I): assign to R the job J at index I (in the sequence of jobs assigned to R) (I >= 1)
%

time(0..n).

% Your encoding please...


%
% show
%
#show at/3.
#show assign/3.


# Formalities.
You can work on the solution alone or in groups of two people. 
Different groups have to submit different solutions, in case
of plagiarism all groups involved will fail the project. 

Your solution should represent correctly all optimal solutions for every instance, 
but you can still pass the project if some instances are not solved within
the time limit.
This is tested automatically by the script ``test.py``. 

We will send you further instructions about the submission process from Moodle.

# Tips:

* You can decompose the encoding in various parts:
  * representing the movement of the robots
  * representing the assignment of jobs to robots
  * checking if the movements and the assignment solve the problem.
  
  For the check part you can consider first the case where there is at most one job per class, 
  as in our example.
  
* Try first to model the simplest instances `simple-*.lp`, and then try one by one 
  the other types of instances.

* Our reference encoding managed to solve all instances except `stop-03.lp`, `stop-04.lp` and `stop-05.lp` in less than a second. It makes sense to try those instances only once you are sure that the others are solved correctly.
  
* Do not try the test script until you are confident that your encoding works well. 
  Debug your encoding calling clingo directly.
  
* If you are stuck you can contact us. We will do out best to answer all your questions. You can send us questions and remarks either via Moodle or by email.

* Start as soon as possible to avoid running out of time. However, if you still realize that you have problems making it before the deadline, please contact us instead of copying another solution.